# Spark Programming Practice

## 0.Basic Operation

### action

In [16]:
x = sc.parallelize(range(1,11),1)
print (x.reduce(lambda x,y : x+y ))
print (x.reduce(lambda x,y : x-y ))
# 1+2+...+10=55
# 1-2-3-4-...-10=-53
x = sc.parallelize(range(1,11),2)
print (x)
print (x.reduce(lambda x,y : x+y ))
print (x.reduce(lambda x,y : x-y ))
# 1+2+3+4+5 + 6+7+8+9+10 = 55
# 1-2-3-4-5 - 6-7-8-9-10 = -13 - -28 = 15

x = sc.parallelize(range(1,11),3)
print (x.reduce(lambda x,y : x+y ))
print (x.reduce(lambda x,y : x-y ))
# 1-4-7

55
-53
PythonRDD[97] at RDD at PythonRDD.scala:48
55
15
55
23


In [22]:
x.takeOrdered(3)

[1, 2, 3]

In [25]:
x.top(3)

for num in x.top(3):
    print (num)

10
9
8


In [43]:
rdd = sc.parallelize([("a",1), ("b", 2), ("a", 3), ("b", 4)], 2)
print (rdd.countByKey())

for num in rdd.countByKey():
    print (num)

print (rdd.collectAsMap())

for num in rdd.collectAsMap():
    print (num)

rdd.lookup("a")

defaultdict(<class 'int'>, {'a': 2, 'b': 2})
a
b
{'a': 3, 'b': 4}
a
b


[1, 3]

In [76]:
# fold
# Aggregate the elements of each partition, and then the results for all the partitions, using a given associative function and a neutral “zero value.”
x = sc.parallelize([1,2,3,4])
neutral_zero_value = 0  # 0 for sum, 1 for multiplication
y = x.fold(neutral_zero_value,lambda obj, accumulated: accumulated + obj) # computes cumulative sum
print(x.collect())
print(y)

x.fold(1, lambda x,y : x*y)

def f(a,b):
    if a> b :
        return a
    else:
        return b
y = x.fold(neutral_zero_value,lambda obj, accumulated: f(obj,accumulated)) # computes cumulative sum
print (y)

[1, 2, 3, 4]
10
4


In [65]:
# Aggregate  聚合每个partition的元素，然后再聚合每个partition
x = sc.parallelize([2,3,4])
neutral_zero_value = (0,1) # sum: x+0 = x, product: 1*x = x
seqOp = (lambda aggregated, el: (aggregated[0] + el, aggregated[1] * el)) 
combOp = (lambda aggregated, el: (aggregated[0] + el[0], aggregated[1] * el[1]))
y = x.aggregate(neutral_zero_value,seqOp,combOp)  # computes (cumulative sum, cumulative product)
print(x.collect())
print(y)

x = sc.parallelize([1,2,3,4], 2)
y = x.aggregate((0,0), (lambda acc, value: (acc[0]+value, acc[1]+1)), (lambda acc1,acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])))
print (y)
x = sc.parallelize([1,2,3,4],3)
y= x.aggregate((1,1), (lambda acc, value: (acc[0]+value, acc[1]+1)), (lambda acc1,acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])))
print (y)


[2, 3, 4]
(9, 24)
(10, 4)
(14, 8)


### transformation

In [103]:
x = sc.parallelize([1,2,3,4],2)
y = x.flatMap(lambda x: (range(x))).collect()
for num in y:
    print (str(num) + " ")

y = x.map(lambda x: (range(x))).collect()
for num in y:
    print (str(num) + " ")
    


rdd = sc.parallelize([1, 2, 3, 4, 5], 4)
def f(iterator): 
    yield sum(iterator)
rdd.mapPartitions(f).collect()



0 
0 
1 
0 
1 
2 
0 
1 
2 
3 
range(0, 1) 
range(0, 2) 
range(0, 3) 
range(0, 4) 


[1, 2, 3, 9]

In [117]:


rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()


6

In [124]:
# sample

x = sc.parallelize((range(100)))
sample = x.sample(withReplacement=True, fraction=0.5).collect()
print (sample)

# takeSample 和sample同发相似，但是第二个参数换了变量，返回的是Collection并非RDD.
x = sc.parallelize(range(7))
ylist = [x.takeSample(withReplacement=False, num=3) for i in range(5)]  # call 'sample' 5 times
print('x = ' + str(x.collect()))
for cnt,y in zip(range(len(ylist)), ylist):
    print('sample:' + str(cnt) + ' y = ' +  str(y))  # no collect on y

[0, 0, 1, 1, 3, 3, 3, 8, 10, 10, 13, 14, 17, 20, 20, 22, 22, 22, 25, 27, 28, 30, 36, 38, 41, 47, 48, 53, 53, 55, 57, 59, 62, 63, 71, 76, 78, 78, 78, 79, 80, 80, 80, 80, 83, 83, 84, 84, 85, 85, 86, 88, 90, 90, 91, 92, 92]


In [133]:
#coalesce
sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(2).collect()
sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(2).glom().collect()

#glom
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()

[[1], [2, 3], [4, 5]]

In [131]:
#random split
rdd = sc.parallelize(range(500), 1)
rdd1, rdd2 = rdd.randomSplit([2, 3], 17)
len(rdd1.collect() + rdd2.collect())
#500
150 < rdd1.count() < 250
#true
250 < rdd2.count() < 350

rdd1.count()

192

In [125]:
# repartition 可以在最初对RDD进行分区操作，这个操作实际上是一个shuffle，可能比较耗时，但是如果之后的action比较多的话，
# 可以减少以后的操作时间，其中N值看这个cpu的个数，一般大于2倍cpu，小于1000

# coalesce 
# 返回一个新的RDD，经过reduce操作后到一个numPartitions 的分区当中,可能需要shuffle
# you go from 1000 partitions to 100 partitions, there will not be a shuffle, 
# instead each of the 100 new partitions will claim 10 of the current partitions.

print (sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect())
print (sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(1).glom().collect())

# glom() 返回一个RDD 用于分组一个RDD到一个list
# glom() flattens elements on the same partition

[[1], [2, 3], [4, 5]]
[[1, 2, 3, 4, 5]]


In [132]:
x = sc.parallelize(['A','A','B'])
y = sc.parallelize(['D','C','A'])
# union 并集
z = x.union(y)
print(z.collect())

# intersection 交集
z = x.intersection(y)
print(z.collect())

# cartesian 笛卡尔积
x = sc.parallelize(['A','B'])
y = sc.parallelize(['C','D'])
z = x.cartesian(y)
print(z.collect())

['A', 'A', 'B', 'D', 'C', 'A']
['A']
[('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D')]


### k-v transformation

In [135]:
#mapValues
x = sc.parallelize([('A',(1,2,3)),('B',(4,5))])
y = x.mapValues(lambda x : [i**2 for i in x])
print (x.collect())
print (y.collect())

[('A', (1, 2, 3)), ('B', (4, 5))]
[('A', [1, 4, 9]), ('B', [16, 25])]


In [138]:
#flatMapValues
x = sc.parallelize([('A',(1,2,3)),('B',(4,5))])
y = x.flatMapValues(lambda x : [i**2 for i in x])
print (x.collect())
print (y.collect())
y = x.flatMapValues(lambda x : list([x,"x"]))
print (y.collect())


[('A', (1, 2, 3)), ('B', (4, 5))]
[('A', 1), ('A', 4), ('A', 9), ('B', 16), ('B', 25)]
[('A', (1, 2, 3)), ('A', 'x'), ('B', (4, 5)), ('B', 'x')]


In [5]:
# combineByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5), ('A', 5), ('C',6)])
createCombiner = (lambda el: [(el,el**2)]) 
mergeVal = (lambda aggregated, el: aggregated + [(el,el**2)]) # append to aggregated
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2 partition之间
y = x.combineByKey(createCombiner,mergeVal,mergeComb)
print(x.collect())
print(y.collect())j

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5), ('A', 5), ('c', 6)]
[('B', [(1, 1), (2, 4)]), ('c', [(6, 36)]), ('A', [(3, 9), (4, 16), (5, 25), (5, 25)])]


In [3]:
# combineByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5), ('A', 5)])
createCombiner = (lambda el: (el,el**2)) 
mergeVal = (lambda aggregated, el: aggregated + (el,el**2)) # append to aggregated
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2 partition之间
y = x.combineByKey(createCombiner,mergeVal,mergeComb)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5), ('A', 5)]
[('B', (1, 1, 2, 4)), ('A', (3, 9, 4, 16, 5, 25, 5, 25))]


In [8]:
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
createCombiner = (lambda x: (x,1))
mergeVal = (lambda aggregated , el : (aggregated[0]+el, aggregated[1]+1))
mergeComb = (lambda agg1, agg2: (agg1[0]+agg2[0],agg1[1]+agg2[1]))
y = x.combineByKey(createCombiner, mergeVal, mergeComb).mapValues(lambda x: x[0]/x[1])
print (y.collect())

[('B', 1.5), ('A', 4.0)]


遍历RDD[(K,V)]中每一个元素

1、如果当前K是一个新元素，使用createCombiner()创建K为键的累加器初始值，生成列表[('A',(3,9))], [('B',(1,1))]

2、如果当前K已经遇到过，使用mergeValue()将当前(K,V)合并进第1步生成的累加器列表,，生成[('A',(3,9)),('A',(4,16)),('A',(5,25)),('B',(1,1,)), ('B', (2,4))],否则执行第1步

3、将相同键值的累加器进行合并，得到[('A', [(3, 9), (4, 16), (5, 25)]), ('B', [(1, 1), (2, 4)])]

因此得到如下结果：

[('A', [(3, 9), (4, 16), (5, 25)]), ('B', [(1, 1), (2, 4)])]

In [2]:
# combineByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5), ('A', 5)], 3)
createCombiner = (lambda el: [el]) 
mergeVal = (lambda aggregated, el: aggregated + [el]) # append to aggregated
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2 partition之间
y = x.combineByKey(createCombiner,mergeVal,mergeComb)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5), ('A', 5)]
[('B', [1, 2]), ('A', [3, 4, 5, 5])]


In [144]:
x = sc.parallelize([("a", 1), ("b", 1), ("a", 2),("a", 2)])
def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a
    
def extend(a, b):
    a.extend(b)
    return a

sorted(x.combineByKey(to_list, append, extend).collect())


[('a', [1, 2, 2]), ('b', [1])]

In [13]:
# foldByKey
x = sc.parallelize([("a", 1), ("b", 1), ("a", 2),("a", 2)])
sorted(x.foldByKey(0, (lambda x,y : x+y)).collect())



[('a', 5), ('b', 1)]

In [14]:
# ReduceBykey
#针对KV结构的数据将相同的key的values放到一起，与groupByKey不同，会进行一个类似mapreduce中的combine操作，减少相应数据的IO操作，加快效率. 
# 如果想进行一些非叠加操作，我们可以将value组合成字符串或其他格式将相同key的values组合在一起，通过迭代组合数据拆开操作.

x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.reduceByKey(lambda agg, obj: agg + obj)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
[('B', 3), ('A', 12)]


In [44]:
# groupByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.groupByKey()
print (y.collect())

for num in y.collect():
    print (list(num))

for key,val in list(y.collect()):
    print(key, val)

[('B', <pyspark.resultiterable.ResultIterable object at 0x7f7e3a2abeb8>), ('A', <pyspark.resultiterable.ResultIterable object at 0x7f7e3a32c710>)]
['B', <pyspark.resultiterable.ResultIterable object at 0x7f7e3a32c978>]
['A', <pyspark.resultiterable.ResultIterable object at 0x7f7e3a32c828>]
B <pyspark.resultiterable.ResultIterable object at 0x7f7e3a32c160>
A <pyspark.resultiterable.ResultIterable object at 0x7f7e3a32c6a0>


In [22]:
# groupBy
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x: x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [29]:
# sortByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.sortByKey(False)
print (y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]


In [30]:
tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
sc.parallelize(tmp2).sortByKey(True, 3, keyfunc=lambda k: k.lower()).collect()


[('a', 3),
 ('fleece', 7),
 ('had', 2),
 ('lamb', 5),
 ('little', 4),
 ('Mary', 1),
 ('was', 8),
 ('white', 9),
 ('whose', 6)]

In [38]:
# groupWith
# 在类型为（K,V)和(K,W)类型的数据集上调用，返回一个数据集，组成元素为（K, Seq[V], Seq[W]) Tuples。
# 这个操作在其它框架上称为CoGroup

x = sc.parallelize([('C',4),('B',(3,3)),('A',2),('A',(1,1))])
y = sc.parallelize([('B',(7,7)),('A',6),('D',(5,5))])
#z = sc.parallelize([('D',9),('B',(8,8))])
#a = x.groupWith(y,z)
a = x.groupWith(y)

print(x.collect())
print(y.collect())
for key,val in list(a.collect()):
    print(key, [list(i) for i in val])


[('C', 4), ('B', (3, 3)), ('A', 2), ('A', (1, 1))]
[('B', (7, 7)), ('A', 6), ('D', (5, 5))]
B [[(3, 3)], [(7, 7)]]
D [[], [(5, 5)]]
C [[4], []]
A [[2, (1, 1)], [6]]


In [53]:
# cogroup
# 对两个RDD中的KV元素，每个RDD中相同key中的元素分别聚合成一个集合，与reduceByKey不同的是cogroup针对两个RDD中相同的key元素进行合并.
# 不同RDD当中的KV元素分别 放在不同的集合当中.

x = sc.parallelize([('C',4),('B',(3,3)),('A',2),('A',(1,1))])
y = sc.parallelize([('A',8),('B',7),('A',6),('D',(5,5))])
z = x.cogroup(y)
print(x.collect())
print(y.collect())
for key,val in list(z.collect()):
    # print(key, [list(i) for i in val])
    print(key, [list(i) for i in val])

[('C', 4), ('B', (3, 3)), ('A', 2), ('A', (1, 1))]
[('A', 8), ('B', 7), ('A', 6), ('D', (5, 5))]
B [[(3, 3)], [7]]
D [[], [(5, 5)]]
C [[4], []]
A [[2, (1, 1)], [8, 6]]


In [55]:
# cogroup
x = sc.parallelize([('a',1),('a',2),('b',3), ('c',1)])
y = sc.parallelize([('a',3),('a',4),('b',5)])

z = x.cogroup(y)
print(x.collect())
print(y.collect())
for key,val in z.collect():
    # print(key, [list(i) for i in val])
    print(key, [list(i) for i in val])

[('a', 1), ('a', 2), ('b', 3), ('c', 1)]
[('a', 3), ('a', 4), ('b', 5)]
b [[3], [5]]
c [[1], []]
a [[1, 2], [3, 4]]


In [56]:
# join(otherDataSet,numPartitions):对两个RDD先进行cogroup操作形成新的RDD，再对每个Key下的元素进行笛卡尔积
x = sc.parallelize([('a',1),('a',2),('b',3), ('c',1)])
y = sc.parallelize([('a',3),('a',4),('b',5)])

z = x.cogroup(y)
print(x.collect())
print(y.collect())
for key,val in z.collect():
    # print(key, [list(i) for i in val])
    print(key, [list(i) for i in val])

z = x.join(y)
print (z.collect())

[('a', 1), ('a', 2), ('b', 3), ('c', 1)]
[('a', 3), ('a', 4), ('b', 5)]
b [[3], [5]]
c [[1], []]
a [[1, 2], [3, 4]]
[('b', (3, 5)), ('a', (1, 3)), ('a', (1, 4)), ('a', (2, 3)), ('a', (2, 4))]


In [59]:
x = sc.parallelize([('a',1),('a',2),('b',3), ('c',1)])
y = sc.parallelize([('a',3),('a',4),('b',5)])

z = x.leftOuterJoin(y)
print (z.collect())

x = sc.parallelize([('a',1),('a',2),('b',3)])
y = sc.parallelize([('a',3),('a',4),('b',5), ('c',1)])
z = x.rightOuterJoin(y)
print (z.collect())

[('b', (3, 5)), ('c', (1, None)), ('a', (1, 3)), ('a', (1, 4)), ('a', (2, 3)), ('a', (2, 4))]
[('b', (3, 5)), ('c', (None, 1)), ('a', (1, 3)), ('a', (1, 4)), ('a', (2, 3)), ('a', (2, 4))]


In [ ]:
# partitionBy
# 可以将RDD进行分区，然后重新生成一个ShuffleRDD,进行一个shuffle操作，对后面进行频繁的shuffle操作可以加快效率.

x = sc.parallelize([(0,1),(1,2),(2,3)],2)
y = x.partitionBy(numPartitions = 3, partitionFunc = lambda x: x)  # only key is passed to paritionFunc
print(x.glom().collect())
print(y.glom().collect())

## practice

In [8]:
x = sc.parallelize(range(7))
ylist = [x.sample(withReplacement=False, fraction=0.5) for i in range(5)] # call 'sample' 5 times
# True表示有放回取样，false表示无放回
print (ylist)
print('x = ' + str(x.collect()))
for cnt,y in zip(range(len(ylist)), ylist):
    print ('sample:' + str(cnt) + ' y = ' +  str(y.collect()))

sample = x.sample(withReplacement=True, fraction=0.5, seed=4).collect()
print (sample)

[PythonRDD[57] at RDD at PythonRDD.scala:48, PythonRDD[58] at RDD at PythonRDD.scala:48, PythonRDD[59] at RDD at PythonRDD.scala:48, PythonRDD[60] at RDD at PythonRDD.scala:48, PythonRDD[61] at RDD at PythonRDD.scala:48]
x = [0, 1, 2, 3, 4, 5, 6]
sample:0 y = [0, 1, 2, 6]
sample:1 y = [5]
sample:2 y = [0, 1, 4, 5]
sample:3 y = [0, 1, 2]
sample:4 y = [0, 1, 2, 4, 5]
[0, 5, 5, 6]


In [76]:
# takeSample 和sample同发相似，但是第二个参数换了变量，返回的是Collection并非RDD.
x = sc.parallelize(range(7))
ylist = [x.takeSample(withReplacement=False, num=3) for i in range(5)]  # call 'sample' 5 times
print('x = ' + str(x.collect()))
for cnt,y in zip(range(len(ylist)), ylist):
    print('sample:' + str(cnt) + ' y = ' +  str(y))  # no collect on y


x = [0, 1, 2, 3, 4, 5, 6]
sample:0 y = [0, 6, 2]
sample:1 y = [0, 4, 6]
sample:2 y = [0, 1, 6]
sample:3 y = [2, 5, 3]
sample:4 y = [5, 3, 4]


In [69]:
x = sc.parallelize(['A','A','B'])
y = sc.parallelize(['D','C','A'])
# union 并集 不去重
z = x.union(y)
print(z.collect())

# intersection 交集 去重
z = x.intersection(y)
print(z.collect())

['A', 'A', 'B', 'D', 'C', 'A']
['A']


In [82]:
# cartesian 笛卡尔积 不去重
x = sc.parallelize(['A','B'])
y = sc.parallelize(['C','D'])
z = x.cartesian(y)
print(z.collect())

[('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D')]


In [110]:
# join
# 相当于inner join. 对两个需要连接的RDD进行cogroup，然后对每个Key下面的list进行笛卡尔积的操作，输出
# 两两相交两个集合作为value ，相当于sql中where a.key= b.key。

x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
sorted(x.join(y).collect())


[('a', (1, 2)), ('a', (1, 3))]

In [113]:
x = sc.parallelize([('C',4),('B',3),('A',2),('A',1)]) 
y = sc.parallelize([('A',8),('B',7),('A',6),('D',5)]) 
z = x.join(y) 
z1 = x.leftOuterJoin(y)
z2 = x.rightOuterJoin(y)

print(z.collect())
print(z1.collect())
print(z2.collect())
print(z.collect())


[('B', (3, 7)), ('A', (2, 8)), ('A', (2, 6)), ('A', (1, 8)), ('A', (1, 6))]
[('B', (3, 7)), ('C', (4, None)), ('A', (2, 8)), ('A', (2, 6)), ('A', (1, 8)), ('A', (1, 6))]
[('B', (3, 7)), ('D', (None, 5)), ('A', (2, 8)), ('A', (2, 6)), ('A', (1, 8)), ('A', (1, 6))]
[('B', (3, 7)), ('A', (2, 8)), ('A', (2, 6)), ('A', (1, 8)), ('A', (1, 6))]


In [88]:
# groupBy
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x: x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])


[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [90]:
# Reduce比groupby好，能用reduce就用reduce
x = sc.parallelize([1,2,3])
y = x.reduce(lambda obj, accumulated: obj + accumulated)  # computes a cumulative sum
print (x)
print(x.collect())
print(y)

ParallelCollectionRDD[221] at parallelize at PythonRDD.scala:489
[1, 2, 3]
6


In [109]:
# fold
# Aggregate the elements of each partition, and then the results for all the partitions, using a given associative function and a neutral “zero value.”
x = sc.parallelize([1,2,3])
neutral_zero_value = 0  # 0 for sum, 1 for multiplication
y = x.fold(neutral_zero_value,lambda obj, accumulated: accumulated + obj) # computes cumulative sum
print(x.collect())
print(y)

[1, 2, 3]
6


In [91]:
# Aggregate  聚合每个partition的元素，然后再聚合每个partition
x = sc.parallelize([2,3,4])
neutral_zero_value = (0,1) # sum: x+0 = x, product: 1*x = x
seqOp = (lambda aggregated, el: (aggregated[0] + el, aggregated[1] * el)) 
combOp = (lambda aggregated, el: (aggregated[0] + el[0], aggregated[1] * el[1]))
y = x.aggregate(neutral_zero_value,seqOp,combOp)  # computes (cumulative sum, cumulative product)
print(x.collect())
print(y)collectAsMap


[2, 3, 4]
(9, 24)


In [98]:
# min，max，sum，count 直接根据单词理解即可
data = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/average.txt")
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).count()
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).sum()

#mean 求平均值 variance方差 stdev偏差
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).mean()
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).variance()
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).stdev()

14.907119849998599

In [106]:
# countByValue 统计元素在element当中出现的频率
x = sc.parallelize([1,3,1,2,3])
y = x.countByValue()
print(x.collect())
print(y)

# countByKey 通过Key进行count
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.countByKey()
print(x.collect())
print(y)




[1, 3, 1, 2, 3]
defaultdict(<class 'int'>, {1: 2, 3: 2, 2: 1})
[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
defaultdict(<class 'int'>, {'B': 2, 'A': 3})


In [102]:
#collectAsMap 返回Key-v结构元素的value

x = sc.parallelize([('C',3),('A',1),('B',2)])
y = x.collectAsMap()
print(x.collect())
print(y)


[('C', 3), ('A', 1), ('B', 2)]
{'C': 3, 'A': 1, 'B': 2}


In [135]:
# KeyBy

# 为RDD当中的elements创建一个元组

x = sc.parallelize([1,2,3])
y = x.keyBy(lambda x: x**2)
print(x.collect())
print(y.collect())

[1, 2, 3]
[(1, 1), (4, 2), (9, 3)]


In [104]:
# Keys 返回一个RDD的keys
m = sc.parallelize([(1, 2), (3, 4)]).keys()
m.collect()

[1, 3]

In [105]:
# Values 返回一个RDD的values

x = sc.parallelize([('C',3),('A',1),('B',2)])
y = x.values()
print(x.collect())
print(y.collect())

[('C', 3), ('A', 1), ('B', 2)]
[3, 1, 2]


In [108]:
# sortByKey
x = sc.parallelize([('B',1),('A',2),('C',3)])
y = x.sortByKey()
print(x.collect())
print(y.collect())

[('B', 1), ('A', 2), ('C', 3)]
[('A', 2), ('B', 1), ('C', 3)]


In [107]:
# ReduceBykey
#针对KV结构的数据将相同的key的values放到一起，与groupByKey不同，会进行一个类似mapreduce中的combine操作，减少相应数据的IO操作，加快效率. 
# 如果想进行一些非叠加操作，我们可以将value组合成字符串或其他格式将相同key的values组合在一起，通过迭代组合数据拆开操作.

x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
y = x.reduceByKey(lambda agg, obj: agg + obj)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
[('B', 3), ('A', 12)]


In [121]:
# partitionBy
# 可以将RDD进行分区，然后重新生成一个ShuffleRDD,进行一个shuffle操作，对后面进行频繁的shuffle操作可以加快效率.

x = sc.parallelize([(0,1),(1,2),(2,3)],2)
y = x.partitionBy(numPartitions = 3, partitionFunc = lambda x: x)  # only key is passed to paritionFunc
print(x.glom().collect())
print(y.glom().collect())

[[(0, 1)], [(1, 2), (2, 3)]]
[[(0, 1)], [(1, 2)], [(2, 3)]]


In [114]:
# combineByKey
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
createCombiner = (lambda el: [(el,el**2)]) 
mergeVal = (lambda aggregated, el: aggregated + [(el,el**2)]) # append to aggregated
mergeComb = (lambda agg1,agg2: agg1 + agg2 )  # append agg1 with agg2
y = x.combineByKey(createCombiner,mergeVal,mergeComb)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
[('B', [(1, 1), (2, 4)]), ('A', [(3, 9), (4, 16), (5, 25)])]


遍历RDD[(K,V)]中每一个元素

1、如果当前K是一个新元素，使用createCombiner()创建K为键的累加器初始值，生成列表[('A',(3,9))], [('B',(1,1))]

2、如果当前K已经遇到过，使用mergeValue()将当前(K,V)合并进第1步生成的累加器列表,，生成[('A',(3,9)),('A',(4,16)),('A',(5,25)),('B',(1,1,)), ('B', (2,4))],否则执行第1步

3、将相同键值的累加器进行合并，得到[('A', [(3, 9), (4, 16), (5, 25)]), ('B', [(1, 1), (2, 4)])]

因此得到如下结果：

[('A', [(3, 9), (4, 16), (5, 25)]), ('B', [(1, 1), (2, 4)])]

In [116]:
# aggregateByKey
# aggregateByKey 把类型为KV的RDD类型转为K U的RDD，V和U的类型可以不一样

# aggregateByKey 内部是通过调用 combineByKey 来实现的，combineByKey 的 createCombine 函数逻辑由 zeroValue 这个变量实现，
# zeroValue 作为聚合的初始值，乘法聚合则为1，集合操作则为空集合
# seqOp在combineByKey中的功能是mergeValues，(U,V)=>U
# combOp在combineByKey中的功能是mergeCombiners

# 从求均值的实现来看，aggregate通过提供零值的方式，避免了combineByKey中的createCombiner步骤
# (createCombiner本质工作就是遇到第一个key时进行初始化操作，这个初始化不是提供零值，而是对第一个(k,v)进行转换得到c的初始值）

x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
zeroValue = [] # empty list is 'zero value' for append operation
seqOp = (lambda aggregated, el: aggregated + [(el,el**2)])
combOp = (lambda agg1,agg2: agg1 + agg2 )
y = x.aggregateByKey(zeroValue,seqOp,combOp)
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
[('B', [(1, 1), (2, 4)]), ('A', [(3, 9), (4, 16), (5, 25)])]


In [117]:
# foldByKey
# foldByKey 合并每一个 key 的所有值，在级联函数和“零值”中使用。
x = sc.parallelize([('B',1),('B',2),('A',3),('A',4),('A',5)])
zeroValue = 1 # one is 'zero value' for multiplication
y = x.foldByKey(zeroValue,lambda agg,x: agg*x )  # computes cumulative product within each key
print(x.collect())
print(y.collect())

[('B', 1), ('B', 2), ('A', 3), ('A', 4), ('A', 5)]
[('B', 2), ('A', 60)]


In [123]:
# mapValues
#　针对KV数据，对数据中的value进行map操作，而不对key进行处理
# 每一个元素的value被输入函数映射为一系列的值，然后这些值再与原RDD中的key组成一系列新的KV对.

x = sc.parallelize([('A',(1,2,3)),('B',(4,5))])
y = x.mapValues(lambda x: [i**2 for i in x]) # function is applied to entire value
print(x.collect())
print(y.collect())

[('A', (1, 2, 3)), ('B', (4, 5))]
[('A', [1, 4, 9]), ('B', [16, 25])]


In [122]:
# flatMapValues

# 类似于把一个[('A', (1, 2, 3)), ('B', (4, 5))] flatmap成以下这种形式
# [('A', 1), ('A', 4), ('A', 9), ('B', 16), ('B', 25)]

x = sc.parallelize([('A',(1,2,3)),('B',(4,5))]) 
y = x.flatMapValues(lambda x: [i**2 for i in x]) 
# function is applied to entire value, then result is flattened 
print(x.collect()) 
print(y.collect())



[('A', (1, 2, 3)), ('B', (4, 5))]
[('A', 1), ('A', 4), ('A', 9), ('B', 16), ('B', 25)]


In [36]:
# groupWith
# 在类型为（K,V)和(K,W)类型的数据集上调用，返回一个数据集，组成元素为（K, Seq[V], Seq[W]) Tuples。
# 这个操作在其它框架上称为CoGroup

x = sc.parallelize([('C',4),('B',(3,3)),('A',2),('A',(1,1))])
y = sc.parallelize([('B',(7,7)),('A',6),('D',(5,5))])
z = sc.parallelize([('D',9),('B',(8,8))])
a = x.groupWith(y,z)
print(x.collect())
print(y.collect())
print(z.collect())
for key,val in list(a.collect()):
    print(key, [list(i) for i in val])


[('C', 4), ('B', (3, 3)), ('A', 2), ('A', (1, 1))]
[('B', (7, 7)), ('A', 6), ('D', (5, 5))]
[('D', 9), ('B', (8, 8))]
B [[(3, 3)], [(7, 7)], [(8, 8)]]
A [[2, (1, 1)], [6], []]
D [[], [(5, 5)], [9]]
C [[4], [], []]


In [125]:
# cogroup
# 对两个RDD中的KV元素，每个RDD中相同key中的元素分别聚合成一个集合，与reduceByKey不同的是cogroup针对两个RDD中相同的key元素进行合并.
# 不同RDD当中的KV元素分别 放在不同的集合当中.

x = sc.parallelize([('C',4),('B',(3,3)),('A',2),('A',(1,1))])
y = sc.parallelize([('A',8),('B',7),('A',6),('D',(5,5))])
z = x.cogroup(y)
print(x.collect())
print(y.collect())
for key,val in list(z.collect()):
    print(key, [list(i) for i in val])

[('C', 4), ('B', (3, 3)), ('A', 2), ('A', (1, 1))]
[('A', 8), ('B', 7), ('A', 6), ('D', (5, 5))]
B [[(3, 3)], [7]]
D [[], [(5, 5)]]
C [[4], []]
A [[2, (1, 1)], [8, 6]]


In [126]:
# sampleByKey
#返回RDD的一个子集合，通过Key采样.

x = sc.parallelize([('A',1),('B',2),('C',3),('B',4),('A',5)])
y = x.sampleByKey(withReplacement=False, fractions={'A':0.5, 'B':1, 'C':0.2})
print(x.collect())
print(y.collect())

[('A', 1), ('B', 2), ('C', 3), ('B', 4), ('A', 5)]
[('A', 1), ('B', 2), ('B', 4)]


In [132]:
# subtractByKey
# 返回KV对类型，x-y 剩下的key-value
x = sc.parallelize([("a", 1), ("b", 4), ("b", 5), ("a", 2),("c",5),("d",6),("e",10)])
y = sc.parallelize([("a", 3), ("c", None),("d",8)])
sorted(x.subtractByKey(y).collect())


[('b', 4), ('b', 5), ('e', 10)]

In [134]:
# subtract
# 对RDD1中取出RDD1于RDD2交集中的所有元素.

x = sc.parallelize([('C',4),('B',3),('A',2),('A',1)])
y = sc.parallelize([('C',8),('A',2),('D',1)])
z = x.subtract(y)
print(x.collect())
print(y.collect())
print(z.collect())

[('C', 4), ('B', 3), ('A', 2), ('A', 1)]
[('C', 8), ('A', 2), ('D', 1)]
[('C', 4), ('A', 1), ('B', 3)]


In [139]:
# repartition 可以在最初对RDD进行分区操作，这个操作实际上是一个shuffle，可能比较耗时，但是如果之后的action比较多的话，
# 可以减少以后的操作时间，其中N值看这个cpu的个数，一般大于2倍cpu，小于1000

# coalesce 
# 返回一个新的RDD，经过reduce操作后到一个numPartitions 的分区当中,可能需要shuffle
# you go from 1000 partitions to 100 partitions, there will not be a shuffle, 
# instead each of the 100 new partitions will claim 10 of the current partitions.

print (sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect())
print (sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(1).glom().collect())

# glom() 返回一个RDD 用于分组一个RDD到一个list
# glom() flattens elements on the same partition

[[1], [2, 3], [4, 5]]
[[1, 2, 3, 4, 5]]


In [147]:
# ZIP
# zipWithUniqueId
# zipWithIndex
# zip expects x and y to have same #partitions and #elements/partition

x = sc.parallelize(['B','A','A'])
y = x.map(lambda x: ord(x))  
z = x.zip(y)
z1 = x.zipWithIndex()
z2 = x.zipWithUniqueId()

print(x.collect())
print(y.collect())
print(z.collect())
print(z1.collect())
print(z2.collect())

['B', 'A', 'A']
[66, 65, 65]
[('B', 66), ('A', 65), ('A', 65)]
[('B', 0), ('A', 1), ('A', 2)]
[('B', 2), ('A', 5), ('A', 7)]


## 1.WordCount

In [1]:
data = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/word.txt")

In [5]:
wordcount = data.flatMap(lambda line : line.split(" ")).map(lambda x: (x,1)).reduceByKey(lambda x,y : x+y)

In [6]:
wordcount.collect()

[('lyc', 1), ('abc', 2), ('xxx', 2), ('bug', 1), ('add', 1), ('fsdf', 1)]

## 2.Average

### (1)

In [7]:
data = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/average.txt")

In [31]:
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x),1)).collect()

[(15, 1), (20, 1), (20, 1), (25, 1), (30, 1), (60, 1)]

In [43]:
average_tuple = data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x),1)).reduce(lambda x,y:(x[0]+y[0],x[1]+y[1]))
print (average_tuple)

(170, 6)


In [41]:
average = average_tuple[0]/average_tuple[1]
print (average)

28.333333333333332


### (2)

In [46]:
print (data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).reduce(lambda x,y : x+y))

170


In [47]:
data.flatMap(lambda line: line.split(" ")).map(lambda x: (int(x))).count()

6

In [48]:
average = 170/6

### (3)

In [12]:
rdd = sc.parallelize([("spark",2),("hadoop",6),("hadoop",4),("spark",6)])

In [13]:
rdd.mapValues(lambda x: (x,1)).collect()

[('spark', (2, 1)), ('hadoop', (6, 1)), ('hadoop', (4, 1)), ('spark', (6, 1))]

In [15]:
rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1])).collect()

[('hadoop', (10, 2)), ('spark', (8, 2))]

In [19]:
rdd.mapValues(lambda x: (x,1)).reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: (x[0]/x[1])).collect()

[('hadoop', 5.0), ('spark', 4.0)]

In [27]:
rdd.countByKey()

defaultdict(int, {'hadoop': 2, 'spark': 2})

In [30]:
rdd.countByValue()

defaultdict(int,
            {('hadoop', 4): 1,
             ('hadoop', 6): 1,
             ('spark', 2): 1,
             ('spark', 6): 1})

### (4) aggregate

In [56]:
data = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/average.txt").flatMap(lambda line: line.split(" ")).map(lambda x: (int(x),1))

In [57]:
print (data.collect())

[(15, 1), (20, 1), (20, 1), (25, 1), (30, 1), (60, 1)]


In [64]:
average_tuple1=data.aggregate((0,0),(lambda acc,value : (acc[0]+value[0],acc[1]+1)),(lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1])))
print (average_tuple1)

(170, 6)


In [61]:
print (average_tuple1[0]/average_tuple1[1])

28.333333333333332


## 3.SparkSQL

Spark创建DataFrame的三种方法

1.Spark中使用toDF函数创建DataFrame

2.Spark中使用createDataFrame函数创建DataFrame

3.通过文件直接创建DataFrame

val df = sqlContext.read.parquet("hdfs:/path/to/file")

val df = spark.read.json("examples/src/main/resources/people.json")

Pyspark

1.使用反射推断schema：

people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

schemaPeople = spark.createDataFrame(people)

schemaPeople.registerTempTable("people")

schemaPeople.createOrReplaceTempView("people")

2.使用编程方式指定

fields = [StructField(field_name1, StringType(), True) , StructField(field_name2, StringType(), True)]

schema = StructType(fields)

schemaPeople = spark.createDataFrame(people, schema)

dataframe实际上就是一张表，可以使用
dataframe.show()

### (1) 创建

In [149]:
data_file = "file:///home/lygbug666/workdir/spark-py-notebooks/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [150]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)

In [151]:
from pyspark import Row
csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)


In [153]:
interactions_df = sqlContext.createDataFrame(row_data)

interactions_df.registerTempTable("interactions")

interactions_df.printSchema()

root
 |-- dst_bytes: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- flag: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- src_bytes: long (nullable = true)



In [154]:
training = spark.createDataFrame([(0, "a b c d e spark", 1.0), 
                                  (1, "b d", 0.0), (2, "spark f g h", 1.0), 
                                  (3, "hadoop mapreduce", 0.0)], 
                                 ["id", "text", "label"])

training.registerTempTable("train")

training.printSchema()

root
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- label: double (nullable = true)



In [162]:
from pyspark.sql.types import *

lines = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/people.txt")
parts = lines.map(lambda l: l.split(","))
# Each line is converted to a tuple.
people = parts.map(lambda p: (p[0], p[1].strip()))

schemaString = "name age"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)

schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")

schemaPeople.show()
results.show()


+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [158]:
lines = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: (p[0], p[1].strip()))

schema1 = StructType([StructField("name", StringType(), True), StructField("age", StringType(), True)])

schemaPeople1 = spark.createDataFrame(people, schema1)

schemaPeople1.createOrReplaceTempView("people1")

results1 = spark.sql("SELECT name FROM people1")
results1.show()


+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [164]:
training1 = spark.createDataFrame([("aaa",1,2),("bbb",3,4),("ccc",3,5),("bbb",4, 6)], 
                                 ["key1", "key2", "key3"])

training1.registerTempTable("train1")

training1.printSchema()

training1.show()

root
 |-- key1: string (nullable = true)
 |-- key2: long (nullable = true)
 |-- key3: long (nullable = true)

+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+



### (2) 选择 select

In [175]:
training1.select("key1","key2", "key3").show()
training1.select("*").show()

sqlContext.sql("select * from train1").show()


+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+



### (3) 过滤 where

In [186]:
# filter 函数

training1.filter(training1.key1>"aaa").show()
training1.filter("key1>'aaa'").show()


sqlContext.sql("select * from train1 where key1 > 'aaa'").show()

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+



In [189]:
# where函数

training1.where(training1.key1>"aaa").show()
training1.where("key1>'aaa'").show()

sqlContext.sql("select * from train1 where key1 > 'aaa'").show()

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

+----+----+----+
|key1|key2|key3|
+----+----+----+
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+



### (4) 分组，聚合，排序

In [276]:
# 分组计数

training1.groupBy("key1").count().sort("key1").show()
training1.groupBy(training1.key1).count().sort(training1.key1.desc()).show()
training1.groupBy("key1").count().sort("count").show()

#上面代码中的count不是记录数，而是对groupBy的聚合结果的计数。

sqlContext.sql("select key1, count(*) as count from train1 group by key1 order by key1 desc").show()
sqlContext.sql("select key1, count(*) as count from train1 group by key1 order by count(*) desc").show()

+----+-----+
|key1|count|
+----+-----+
| aaa|    1|
| bbb|    2|
| ccc|    1|
+----+-----+

+----+-----+
|key1|count|
+----+-----+
| ccc|    1|
| bbb|    2|
| aaa|    1|
+----+-----+

+----+-----+
|key1|count|
+----+-----+
| ccc|    1|
| aaa|    1|
| bbb|    2|
+----+-----+

+----+-----+
|key1|count|
+----+-----+
| ccc|    1|
| bbb|    2|
| aaa|    1|
+----+-----+

+----+-----+
|key1|count|
+----+-----+
| bbb|    2|
| ccc|    1|
| aaa|    1|
+----+-----+



In [205]:
# 如果是要看分组后有多少条记录，代码如下
training1.select("key1").distinct().show()
sqlContext.sql("select distinct key1 from train1").show()


training1.select("key1").distinct().count()
sqlContext.sql("select count(distinct key1) from train1").show()


+----+
|key1|
+----+
| ccc|
| aaa|
| bbb|
+----+

+----+
|key1|
+----+
| ccc|
| aaa|
| bbb|
+----+

+--------------------+
|count(DISTINCT key1)|
+--------------------+
|                   3|
+--------------------+



In [228]:
training1.groupBy("key1").agg({'key2': 'mean'}).collect()
training1.groupBy("key1").agg({'key2': 'mean'}).show()

training1.groupBy("key1").agg({'key2': 'max'}).show()
training1.groupBy().avg().show()

training1.groupBy(["key1", training1.key2]).count().show()


+----+---------+
|key1|avg(key2)|
+----+---------+
| ccc|      3.0|
| aaa|      1.0|
| bbb|      3.5|
+----+---------+

+----+---------+
|key1|max(key2)|
+----+---------+
| ccc|        3|
| aaa|        1|
| bbb|        4|
+----+---------+

+---------+---------+
|avg(key2)|avg(key3)|
+---------+---------+
|     2.75|     4.25|
+---------+---------+

+----+----+-----+
|key1|key2|count|
+----+----+-----+
| aaa|   1|    1|
| ccc|   3|    1|
| bbb|   4|    1|
| bbb|   3|    1|
+----+----+-----+



In [219]:
sqlContext.sql("select key1, count(key1), max(key2), avg(key3) from train1 group by key1").show()


+----+-----------+---------+---------+
|key1|count(key1)|max(key2)|avg(key3)|
+----+-----------+---------+---------+
| ccc|          1|        3|      5.0|
| aaa|          1|        1|      2.0|
| bbb|          2|        4|      5.0|
+----+-----------+---------+---------+



### (5)多表操作

In [233]:
df1 = spark.createDataFrame([("aaa", 1, 2), ("bbb", 3, 4), ("ccc", 3, 5), ("bbb", 4, 6)], 
                                 ["key1", "key2", "key3"])

df1.registerTempTable("df1")

df1.printSchema()

df1.show()

df2 = spark.createDataFrame([("aaa", 2, 2),("bbb", 3, 5),("ddd", 3, 5),("bbb", 4, 6),("eee", 1, 2),("aaa", 1, 5),("fff",5,6)], 
                                 ["key1", "key2", "key4"])

df2.registerTempTable("df2")

df2.printSchema()

df2.show()

root
 |-- key1: string (nullable = true)
 |-- key2: long (nullable = true)
 |-- key3: long (nullable = true)

+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
+----+----+----+

root
 |-- key1: string (nullable = true)
 |-- key2: long (nullable = true)
 |-- key4: long (nullable = true)

+----+----+----+
|key1|key2|key4|
+----+----+----+
| aaa|   2|   2|
| bbb|   3|   5|
| ddd|   3|   5|
| bbb|   4|   6|
| eee|   1|   2|
| aaa|   1|   5|
| fff|   5|   6|
+----+----+----+



In [239]:
df3 = df1.join(df2, "key1")
df3.show()

sqlContext.sql("select * from df1, df2 where df1.key1 = df2.key1").show()

sqlContext.sql("select * from df1 join df2 on df1.key1 = df2.key1").show()

+----+----+----+----+----+
|key1|key2|key3|key2|key4|
+----+----+----+----+----+
| aaa|   1|   2|   2|   2|
| aaa|   1|   2|   1|   5|
| bbb|   3|   4|   3|   5|
| bbb|   3|   4|   4|   6|
| bbb|   4|   6|   3|   5|
| bbb|   4|   6|   4|   6|
+----+----+----+----+----+

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   2|   2|
| aaa|   1|   2| aaa|   1|   5|
| bbb|   3|   4| bbb|   3|   5|
| bbb|   3|   4| bbb|   4|   6|
| bbb|   4|   6| bbb|   3|   5|
| bbb|   4|   6| bbb|   4|   6|
+----+----+----+----+----+----+

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   2|   2|
| aaa|   1|   2| aaa|   1|   5|
| bbb|   3|   4| bbb|   3|   5|
| bbb|   3|   4| bbb|   4|   6|
| bbb|   4|   6| bbb|   3|   5|
| bbb|   4|   6| bbb|   4|   6|
+----+----+----+----+----+----+



In [252]:
df4 = df1.join(df2, "key1", "outer").select(df1.key1,df1.key2,df1.key3).collect()
df4 = df1.join(df2, "key1", "outer")

df4.show()

sqlContext.sql("select * from df1 outer join df2 on df1.key1 = df2.key1").show()

AnalysisException: 'resolved attribute(s) key1#1629 missing from key2#1643L,key4#1644L,key3#1631L,key2#1630L,key1#1835 in operator !Project [key1#1629, key2#1630L, key3#1631L];;\n!Project [key1#1629, key2#1630L, key3#1631L]\n+- Project [coalesce(key1#1629, key1#1642) AS key1#1835, key2#1630L, key3#1631L, key2#1643L, key4#1644L]\n   +- Join FullOuter, (key1#1629 = key1#1642)\n      :- LogicalRDD [key1#1629, key2#1630L, key3#1631L]\n      +- LogicalRDD [key1#1642, key2#1643L, key4#1644L]\n'

In [257]:
df5 = df1.join(df2, "key1", "left_outer")
df5.show()

sqlContext.sql("select * from df1 left outer join df2 on df1.key1 = df2.key1 order by df1.key1").show()

+----+----+----+----+----+
|key1|key2|key3|key2|key4|
+----+----+----+----+----+
| ccc|   3|   5|null|null|
| aaa|   1|   2|   2|   2|
| aaa|   1|   2|   1|   5|
| bbb|   3|   4|   3|   5|
| bbb|   3|   4|   4|   6|
| bbb|   4|   6|   3|   5|
| bbb|   4|   6|   4|   6|
+----+----+----+----+----+

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   2|   2|
| aaa|   1|   2| aaa|   1|   5|
| bbb|   3|   4| bbb|   3|   5|
| bbb|   4|   6| bbb|   3|   5|
| bbb|   3|   4| bbb|   4|   6|
| bbb|   4|   6| bbb|   4|   6|
| ccc|   3|   5|null|null|null|
+----+----+----+----+----+----+



In [260]:
df6 = df1.join(df2, "key1", "right_outer").sort(df2.key1)
df6.show()

sqlContext.sql("select * from df1 right outer join df2 on df1.key1 = df2.key1 order by df2.key1").show()

+----+----+----+----+----+
|key1|key2|key3|key2|key4|
+----+----+----+----+----+
| aaa|   1|   2|   2|   2|
| aaa|   1|   2|   1|   5|
| bbb|   3|   4|   3|   5|
| bbb|   3|   4|   4|   6|
| bbb|   4|   6|   4|   6|
| bbb|   4|   6|   3|   5|
| ddd|null|null|   3|   5|
| eee|null|null|   1|   2|
| fff|null|null|   5|   6|
+----+----+----+----+----+

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   2|   2|
| aaa|   1|   2| aaa|   1|   5|
| bbb|   3|   4| bbb|   4|   6|
| bbb|   4|   6| bbb|   3|   5|
| bbb|   4|   6| bbb|   4|   6|
| bbb|   3|   4| bbb|   3|   5|
|null|null|null| ddd|   3|   5|
|null|null|null| eee|   1|   2|
|null|null|null| fff|   5|   6|
+----+----+----+----+----+----+



In [261]:
df7 = df1.join(df2, "key1", "left_Semi")
df7.show()


+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| bbb|   4|   6|
+----+----+----+



In [262]:
df8 = df1.crossJoin(df2)
df8.show(10)

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   2|   2|
| aaa|   1|   2| bbb|   3|   5|
| aaa|   1|   2| ddd|   3|   5|
| aaa|   1|   2| bbb|   4|   6|
| aaa|   1|   2| eee|   1|   2|
| aaa|   1|   2| aaa|   1|   5|
| aaa|   1|   2| fff|   5|   6|
| bbb|   3|   4| aaa|   2|   2|
| bbb|   3|   4| bbb|   3|   5|
| bbb|   3|   4| ddd|   3|   5|
+----+----+----+----+----+----+
only showing top 10 rows



In [270]:
df9 = df1.join(df2, ["key1", "key2"])
df9.show()

sqlContext.sql("select * from df1 join df2 on df1.key1=df2.key1 and df1.key2=df2.key2").show()

+----+----+----+----+
|key1|key2|key3|key4|
+----+----+----+----+
| aaa|   1|   2|   5|
| bbb|   4|   6|   6|
| bbb|   3|   4|   5|
+----+----+----+----+

+----+----+----+----+----+----+
|key1|key2|key3|key1|key2|key4|
+----+----+----+----+----+----+
| aaa|   1|   2| aaa|   1|   5|
| bbb|   4|   6| bbb|   4|   6|
| bbb|   3|   4| bbb|   3|   5|
+----+----+----+----+----+----+



In [273]:
df10 = df1.union(df2)

df10.show()

df10.rdd.getNumPartitions()

+----+----+----+
|key1|key2|key3|
+----+----+----+
| aaa|   1|   2|
| bbb|   3|   4|
| ccc|   3|   5|
| bbb|   4|   6|
| aaa|   2|   2|
| bbb|   3|   5|
| ddd|   3|   5|
| bbb|   4|   6|
| eee|   1|   2|
| aaa|   1|   5|
| fff|   5|   6|
+----+----+----+



16

## 4.matrix multiplication

## 5.Inverted index

### 5.1 Inverted_index in many file

In [12]:
data = sc.wholeTextFiles("file:///home/lygbug666/workdir/spark-py-notebooks/inverted_index")
data.collect()

[('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id5',
  'hadoop is good in world\nspark is very famous in world\nI am rabbit\nyou are cat\n'),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id3',
  'hello cat\nhello rabbit\ncat is doing spark\ncat is the world\nspark is good\n'),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id2',
  'hello spark\nthe spark app for you\nI love you\nyou are a cat\n'),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id4',
  'spark is better than hadoop\nI love hadoop\nyou love spark\ncat and rabbit love app\n'),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id1',
  'hello world\nhello hadoop\nhadoop love\nlove cat\ncat love rabbit\n')]

In [13]:
file_name = data.mapValues(lambda line: line.split(" "))
file_name.collect()

[('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id5',
  ['hadoop',
   'is',
   'good',
   'in',
   'world\nspark',
   'is',
   'very',
   'famous',
   'in',
   'world\nI',
   'am',
   'rabbit\nyou',
   'are',
   'cat\n']),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id3',
  ['hello',
   'cat\nhello',
   'rabbit\ncat',
   'is',
   'doing',
   'spark\ncat',
   'is',
   'the',
   'world\nspark',
   'is',
   'good\n']),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id2',
  ['hello',
   'spark\nthe',
   'spark',
   'app',
   'for',
   'you\nI',
   'love',
   'you\nyou',
   'are',
   'a',
   'cat\n']),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id4',
  ['spark',
   'is',
   'better',
   'than',
   'hadoop\nI',
   'love',
   'hadoop\nyou',
   'love',
   'spark\ncat',
   'and',
   'rabbit',
   'love',
   'app\n']),
 ('file:/home/lygbug666/workdir/spark-py-notebooks/inverted_index/id1',
  ['hello',
   'world\nhel

### 5.2 Inverted_index in one file

In [36]:
data1 = sc.textFile("file:///home/lygbug666/workdir/spark-py-notebooks/inverted_index/input")
data1.collect()

['doc1\tApache Spark Scala Hadoop Java C Python Do And Will KNN',
 'doc2\tSVM Scala News Play Akka Yes GBDT',
 'doc3\tLDA SVM RF GBDT Adaboost Kmeans KNN',
 'doc4\tQQ BAT I Great All LDA',
 'doc5\tApache Hadoop MapReduce Git SVN SVM']

In [37]:
data1.map(lambda line: line.split("\t")).collect()

[['doc1', 'Apache Spark Scala Hadoop Java C Python Do And Will KNN'],
 ['doc2', 'SVM Scala News Play Akka Yes GBDT'],
 ['doc3', 'LDA SVM RF GBDT Adaboost Kmeans KNN'],
 ['doc4', 'QQ BAT I Great All LDA'],
 ['doc5', 'Apache Hadoop MapReduce Git SVN SVM']]

In [38]:
data1.map(lambda line: line.split("\t")).map(lambda x: (x[0],x[1])).collect()

[('doc1', 'Apache Spark Scala Hadoop Java C Python Do And Will KNN'),
 ('doc2', 'SVM Scala News Play Akka Yes GBDT'),
 ('doc3', 'LDA SVM RF GBDT Adaboost Kmeans KNN'),
 ('doc4', 'QQ BAT I Great All LDA'),
 ('doc5', 'Apache Hadoop MapReduce Git SVN SVM')]

In [47]:
index = data1.map(lambda line: line.split("\t")).map(lambda x: (x[0],x[1].split(" ")))
print (index.collect())

# 传统的正排索引

[('doc1', ['Apache', 'Spark', 'Scala', 'Hadoop', 'Java', 'C', 'Python', 'Do', 'And', 'Will', 'KNN']), ('doc2', ['SVM', 'Scala', 'News', 'Play', 'Akka', 'Yes', 'GBDT']), ('doc3', ['LDA', 'SVM', 'RF', 'GBDT', 'Adaboost', 'Kmeans', 'KNN']), ('doc4', ['QQ', 'BAT', 'I', 'Great', 'All', 'LDA']), ('doc5', ['Apache', 'Hadoop', 'MapReduce', 'Git', 'SVN', 'SVM'])]


In [66]:
inverted_word = index.map(lambda x: (x[1],x[0]))

print (inverted_index.collect())

[(['Apache', 'Spark', 'Scala', 'Hadoop', 'Java', 'C', 'Python', 'Do', 'And', 'Will', 'KNN'], 'doc1'), (['SVM', 'Scala', 'News', 'Play', 'Akka', 'Yes', 'GBDT'], 'doc2'), (['LDA', 'SVM', 'RF', 'GBDT', 'Adaboost', 'Kmeans', 'KNN'], 'doc3'), (['QQ', 'BAT', 'I', 'Great', 'All', 'LDA'], 'doc4'), (['Apache', 'Hadoop', 'MapReduce', 'Git', 'SVN', 'SVM'], 'doc5')]


In [67]:
def f(x):
    list = []
    for word in x[0]:
        list.append((word,x[1]))
    return list

In [68]:
inverted_wordllist = inverted_word.map(lambda x: f(x))

print (inverted_wordllist.collect())

[[('Apache', 'doc1'), ('Spark', 'doc1'), ('Scala', 'doc1'), ('Hadoop', 'doc1'), ('Java', 'doc1'), ('C', 'doc1'), ('Python', 'doc1'), ('Do', 'doc1'), ('And', 'doc1'), ('Will', 'doc1'), ('KNN', 'doc1')], [('SVM', 'doc2'), ('Scala', 'doc2'), ('News', 'doc2'), ('Play', 'doc2'), ('Akka', 'doc2'), ('Yes', 'doc2'), ('GBDT', 'doc2')], [('LDA', 'doc3'), ('SVM', 'doc3'), ('RF', 'doc3'), ('GBDT', 'doc3'), ('Adaboost', 'doc3'), ('Kmeans', 'doc3'), ('KNN', 'doc3')], [('QQ', 'doc4'), ('BAT', 'doc4'), ('I', 'doc4'), ('Great', 'doc4'), ('All', 'doc4'), ('LDA', 'doc4')], [('Apache', 'doc5'), ('Hadoop', 'doc5'), ('MapReduce', 'doc5'), ('Git', 'doc5'), ('SVN', 'doc5'), ('SVM', 'doc5')]]


In [75]:
inverted_index = inverted_wordllist.flatMap(lambda x: x)
print (inverted_index.collect())

[('Apache', 'doc1'), ('Spark', 'doc1'), ('Scala', 'doc1'), ('Hadoop', 'doc1'), ('Java', 'doc1'), ('C', 'doc1'), ('Python', 'doc1'), ('Do', 'doc1'), ('And', 'doc1'), ('Will', 'doc1'), ('KNN', 'doc1'), ('SVM', 'doc2'), ('Scala', 'doc2'), ('News', 'doc2'), ('Play', 'doc2'), ('Akka', 'doc2'), ('Yes', 'doc2'), ('GBDT', 'doc2'), ('LDA', 'doc3'), ('SVM', 'doc3'), ('RF', 'doc3'), ('GBDT', 'doc3'), ('Adaboost', 'doc3'), ('Kmeans', 'doc3'), ('KNN', 'doc3'), ('QQ', 'doc4'), ('BAT', 'doc4'), ('I', 'doc4'), ('Great', 'doc4'), ('All', 'doc4'), ('LDA', 'doc4'), ('Apache', 'doc5'), ('Hadoop', 'doc5'), ('MapReduce', 'doc5'), ('Git', 'doc5'), ('SVN', 'doc5'), ('SVM', 'doc5')]


In [76]:
inverted_index1 = inverted_index.reduceByKey(lambda x,y : x+ "|" +y)
print (inverted_index1.collect())

[('Apache', 'doc1|doc5'), ('Spark', 'doc1'), ('Java', 'doc1'), ('C', 'doc1'), ('Python', 'doc1'), ('Do', 'doc1'), ('And', 'doc1'), ('Will', 'doc1'), ('News', 'doc2'), ('QQ', 'doc4'), ('Great', 'doc4'), ('MapReduce', 'doc5'), ('SVN', 'doc5'), ('Scala', 'doc1|doc2'), ('Hadoop', 'doc1|doc5'), ('KNN', 'doc1|doc3'), ('SVM', 'doc2|doc3|doc5'), ('Play', 'doc2'), ('Akka', 'doc2'), ('Yes', 'doc2'), ('GBDT', 'doc2|doc3'), ('LDA', 'doc3|doc4'), ('RF', 'doc3'), ('Adaboost', 'doc3'), ('Kmeans', 'doc3'), ('BAT', 'doc4'), ('I', 'doc4'), ('All', 'doc4'), ('Git', 'doc5')]


In [78]:
for index in inverted_index1.collect() :
    print (index)

('Apache', 'doc1|doc5')
('Spark', 'doc1')
('Java', 'doc1')
('C', 'doc1')
('Python', 'doc1')
('Do', 'doc1')
('And', 'doc1')
('Will', 'doc1')
('News', 'doc2')
('QQ', 'doc4')
('Great', 'doc4')
('MapReduce', 'doc5')
('SVN', 'doc5')
('Scala', 'doc1|doc2')
('Hadoop', 'doc1|doc5')
('KNN', 'doc1|doc3')
('SVM', 'doc2|doc3|doc5')
('Play', 'doc2')
('Akka', 'doc2')
('Yes', 'doc2')
('GBDT', 'doc2|doc3')
('LDA', 'doc3|doc4')
('RF', 'doc3')
('Adaboost', 'doc3')
('Kmeans', 'doc3')
('BAT', 'doc4')
('I', 'doc4')
('All', 'doc4')
('Git', 'doc5')
